# LLMPerf 基本使用

## 安装

In [1]:
%%time
# %%capture

!rm -rf llmperf

!export http_proxy="http://myproxy:7890"
!export https_proxy="http://myproxy:7890"

# 克隆仓库
!git clone https://github.com/ray-project/llmperf.git

# 进入仓库目录
%cd llmperf

!touch setup.cfg

# 安装依赖
!pip install -e .

Cloning into 'llmperf'...
remote: Enumerating objects: 159, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 159 (delta 24), reused 21 (delta 20), pack-reused 116
Receiving objects: 100% (159/159), 247.07 KiB | 634.00 KiB/s, done.
Resolving deltas: 100% (76/76), done.
/root/notebook/my-jupyter-notebook/llm/llmperf/llmperf


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Obtaining file:///root/notebook/my-jupyter-notebook/llm/llmperf/llmperf
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Running setup.py develop for UNKNOWN
CPU times: user 140 ms, sys: 42.4 ms, total: 182 ms
Wall time: 13.7 s


In [2]:
%%time

# https://huggingface.co/hf-internal-testing/llama-tokenizer/tree/main
# 从下载的目录复制
!mkdir hf-internal-testing
!cp -r /models/llama-tokenizer hf-internal-testing
# # 或者设置代理代码会自动下载
# %env http_proxy="http:/myproxy:7890"
# %env https_proxy="http:/myproxy:7890"

CPU times: user 993 µs, sys: 8.05 ms, total: 9.04 ms
Wall time: 266 ms


## 负载测试

In [3]:
%%time

# 设置环境变量
%env OPENAI_API_KEY=ollama
%env OPENAI_API_BASE=http://monkey:11434/v1

# !python3 token_benchmark_ray.py \
!PYTHONPATH=$(pwd)/src python3 token_benchmark_ray.py \
--model "qwen2" \
--mean-input-tokens 550 \
--stddev-input-tokens 150 \
--mean-output-tokens 150 \
--stddev-output-tokens 10 \
--max-num-completed-requests 2 \
--timeout 600 \
--num-concurrent-requests 1 \
--results-dir "result_outputs" \
--llm-api openai \
--additional-sampling-params '{}'


env: OPENAI_API_KEY=ollama
env: OPENAI_API_BASE=http://monkey:11434/v1
2024-07-29 11:34:52,671	INFO worker.py:1781 -- Started a local Ray instance.
100%|█████████████████████████████████████████████| 2/2 [00:07<00:00,  3.84s/it]
\Results for token benchmark for qwen2 queried with the openai api.

inter_token_latency_s
    p25 = 0.017874787592801773
    p50 = 0.019646871597698498
    p75 = 0.021418955602595224
    p90 = 0.02248220600553326
    p95 = 0.022836622806512604
    p99 = 0.02312015624729608
    mean = 0.019646871597698498
    min = 0.016102703587905047
    max = 0.02319103960749195
    stddev = 0.005012210466778759
ttft_s
    p25 = 0.5501839839853346
    p50 = 0.8344089183956385
    p75 = 1.1186338528059423
    p90 = 1.2891688134521246
    p95 = 1.3460138003341853
    p99 = 1.391489789839834
    mean = 0.8344089183956385
    min = 0.2659590495750308
    max = 1.4028587872162461
    stddev = 0.8039095140153102
end_to_end_latency_s
    p25 = 3.267872414784506
    p50 = 3.50826560

## 正确性测试

In [4]:
# 设置环境变量
%env OPENAI_API_KEY=ollama
%env OPENAI_API_BASE=http://monkey:11434/v1

!PYTHONPATH=$(pwd)/src python3 llm_correctness.py \
--model "qwen2" \
--max-num-completed-requests 150 \
--timeout 600 \
--num-concurrent-requests 10 \
--results-dir "result_outputs"

env: OPENAI_API_KEY=ollama
env: OPENAI_API_BASE=http://monkey:11434/v1
2024-07-29 11:35:06,702	INFO worker.py:1781 -- Started a local Ray instance.
100%|█████████████████████████████████████████| 150/150 [00:11<00:00, 13.35it/s]
Mismatched and errored requests.
    mismatched request: 8800, expected: 8880

Results for llm correctness test for qwen2 queried with the openai api.
Errors: 0, Error rate: 0.0
Mismatched: 1, Mismatch rate: 0.006666666666666667
Completed: 150
Completed without errors: 150
